In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
import sys, os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import math

In [4]:
sys.path.append(os.path.realpath('..'))

In [33]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
xgb_train_df = pd.read_csv(r'rendered_data/XGB_train_submission.csv', index_col=0)
lgbm_train_df = pd.read_csv(r'rendered_data/LGBM_train_submission.csv', index_col=0)
cat_train_df = pd.read_csv(r'rendered_data/CAT_train_submission.csv', index_col=0)

xgb_test_df = pd.read_csv(r'rendered_data/XGB_test_submission.csv', index_col=0)
lgbm_test_df = pd.read_csv(r'rendered_data/LGBM_test_submission.csv', index_col=0)
cat_test_df = pd.read_csv(r'rendered_data/CAT_test_submission.csv', index_col=0)

In [30]:
def roc_auc_score(y_valid, preds):
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)
    return roc_auc

In [31]:
combos = []

for x in range(100):
    for y in range(100):
        if x + y < 100:
            z = 100 - x - y
            combos.append([x, y, z])
combos = np.array(combos) * ([[0.01]*3]*len(combos))
len(combos)

5050

In [32]:
coefficients = combos
best_score = 0
best_coefficients_index = 0

for index, (a, b, c) in tqdm(enumerate(coefficients)):
    score = roc_auc_score(train_df["stroke"], (a * xgb_df['stroke']) + (b * lgbm_df['stroke']) + (c * cat_df['stroke']))
    if score > best_score:
        best_score = score
        best_coefficients_index = index

5050it [00:09, 532.56it/s]


In [35]:
coef = combos[best_coefficients_index]
blended_test_results = xgb_test_df['stroke']*coef[0] + lgbm_test_df['stroke']*coef[1] + cat_test_df['stroke']*coef[2]
final_blended_test_submission = pd.DataFrame(data={'id': test_df.index, 'stroke': blended_test_results})
final_blended_test_submission.to_csv(fr'rendered_data/final_blended_test_submission.csv', index=False)